In [6]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

print(" PYTHONPATH set to:", project_root)


 PYTHONPATH set to: C:\Users\Илья\Desktop\interp_dev


In [29]:
os.chdir("C:/Users/Илья/Desktop/interp_dev")  #установка вручную
print("Working dir:", os.getcwd())

Working dir: C:\Users\Илья\Desktop\interp_dev


In [10]:
import torch
from torchinfo import summary
from formants.transformer.layer_norm_ada import AdaptiveLayerNorm
from formants.transformer.multihead_self_attention_ada import MultiHeadSelfAttention
from formants.transformer.feed_forward_ada import FeedForwardAda
from formants.transformer.transformer_Block_AdaLN import TransformerBlockAdaLN
from formants.transformer.mlp_adaln import MlpAdaLN
from formants.tokenizer.phoneme_tokenizer import PhonemeTokenizer

In [ ]:
#ДАННЫЙ НОУТБУК ВОЗМОЖНО ТРЕБУЕТ ПЕРЕРАБОТКИ

In [12]:
x = torch.randn(2, 5, 512)  
alpha = torch.ones(1, 1, 512)
beta = torch.zeros(1, 1, 512)
gamma = torch.ones(1, 1, 512)

In [14]:
ln = AdaptiveLayerNorm(512)
out_ln = ln(x, alpha, beta)
print("AdaLN out shape:", out_ln.shape)

AdaLN out shape: torch.Size([2, 5, 512])


In [16]:
attn = MultiHeadSelfAttention(512, num_heads=8)
out_attn = attn(x, gamma)
print("Attention out shape:", out_attn.shape)

Attention out shape: torch.Size([2, 5, 512])


In [18]:
ff = FeedForwardAda(512, d_ff=1024, dropout=0.1)
out_ff = ff(x, gamma)
print("FF out shape:", out_ff.shape)


FF out shape: torch.Size([2, 5, 512])


In [20]:
hidden_dim = 512
speech_embedding = torch.randn(1, 256)

mlp = MlpAdaLN(input_dim=256, hidden_dim=hidden_dim)
alpha1, beta1, gamma1, alpha2, beta2, gamma2 = mlp(speech_embedding)

print("AdaLN params shapes:")
print(alpha1.shape, beta1.shape, gamma1.shape, alpha2.shape, beta2.shape, gamma2.shape)


AdaLN params shapes:
torch.Size([1, 512]) torch.Size([1, 512]) torch.Size([1, 512]) torch.Size([1, 512]) torch.Size([1, 512]) torch.Size([1, 512])


In [22]:
x1 = torch.randn(1, 5, hidden_dim)
block = TransformerBlockAdaLN(hidden_dim=hidden_dim, dropout=0.1)

alpha1_ = alpha1.unsqueeze(1).expand_as(x1)
beta1_  = beta1.unsqueeze(1).expand_as(x1)
gamma1_ = gamma1.unsqueeze(1).expand_as(x1)
alpha2_ = alpha2.unsqueeze(1).expand_as(x1)
beta2_  = beta2.unsqueeze(1).expand_as(x1)
gamma2_ = gamma2.unsqueeze(1).expand_as(x1)

out1 = block(x1, alpha1_, beta1_, gamma1_, alpha2_, beta2_, gamma2_)
print("Block output (random input):", out1.shape)


Block output (random input): torch.Size([1, 5, 512])


In [31]:
vocab_path = "formants/tokenizer/phoneme_vocab.json"
tokenizer = PhonemeTokenizer(vocab_path)

phonemes = ['HH', 'AH0', 'L', 'OW1', 'W']
token_ids = tokenizer.encode(phonemes)
print("Token IDs:", token_ids)

tokens_tensor = torch.tensor(token_ids).unsqueeze(0)  


Token IDs: [35, 8, 44, 49, 67]


In [37]:
embedding = torch.nn.Embedding(num_embeddings=len(tokenizer.vocab), embedding_dim=hidden_dim)
x2 = embedding(tokens_tensor) 

alpha1_ = alpha1.unsqueeze(1).expand_as(x2)
beta1_  = beta1.unsqueeze(1).expand_as(x2)
gamma1_ = gamma1.unsqueeze(1).expand_as(x2)
alpha2_ = alpha2.unsqueeze(1).expand_as(x2)
beta2_  = beta2.unsqueeze(1).expand_as(x2)
gamma2_ = gamma2.unsqueeze(1).expand_as(x2)

out2 = block(x2, alpha1_, beta1_, gamma1_, alpha2_, beta2_, gamma2_)
print("Block output (token input):", out2.shape)


Block output (token input): torch.Size([1, 5, 512])


In [35]:
summary(
    block,
    input_data=(x2, alpha1_, beta1_, gamma1_, alpha2_, beta2_, gamma2_),
    col_names=["input_size", "output_size", "num_params"],
    depth=2
)


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
TransformerBlockAdaLN                    [1, 5, 512]               [1, 5, 512]               --
├─AdaptiveLayerNorm: 1-1                 [1, 5, 512]               [1, 5, 512]               --
├─MultiHeadSelfAttention: 1-2            [1, 5, 512]               [1, 5, 512]               --
│    └─Linear: 2-1                       [1, 5, 512]               [1, 5, 1536]              787,968
│    └─Dropout: 2-2                      [1, 8, 5, 5]              [1, 8, 5, 5]              --
│    └─Linear: 2-3                       [1, 5, 512]               [1, 5, 512]               262,656
├─AdaptiveLayerNorm: 1-3                 [1, 5, 512]               [1, 5, 512]               --
├─FeedForwardAda: 1-4                    [1, 5, 512]               [1, 5, 512]               --
│    └─Linear: 2-4                       [1, 5, 512]               [1, 5, 2048]              1,050,624
│    └─Dropout: 2-